In [45]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


In [0]:
import numpy as np
import pandas as pd

# Functions

In [0]:
def series_upper(series):
    ## change all strings to upper
    return series.str.upper()

def read_data(n): 
    ## input is constituency no.
    ## have to change it into the real no in the table
    ## starting from 0 sheet
    real_no=int((n-1)/2)
    data=pd.read_excel(file,sheet_name=real_no)
    data.fillna("None",inplace=True)
    return data

def get_alliance(data,alliance_dict):
    alliances={}
    count=0
    for party in data.loc[1,data.loc[1,]!="None"].str.upper():
        count+=1
        for alliance, parties in alliance_dict.items():
            for p in parties.values():
                if party in p:
                    alliances[count]=alliance
        if count not in alliances.keys():
            alliances[count]="Others"
    alliances_=list(alliances.values())
    return alliances_

def get_ally_votes(data,alliance_names,alliance_dict):
    alliances=get_alliance(data,alliance_dict)

    ## get party name in each alliance
    parties=get_ally_party(data,alliance_dict)
    party_row=[]
    for alliance in alliance_names:
        if len(parties[alliance])<2:
            party_row.append(str(parties[alliance]).strip("[").strip("]").strip("'"))
        else:
            party_row.append(parties[alliance])
    party_row=pd.Series(party_row).to_frame().T
    party_row.columns=alliance_names

    ## output dataframe
    temp_df=pd.DataFrame()
    temp_df=data.loc[2:len(data)-4,data.loc[1,]!="None"]
    temp_df.columns=alliances
    temp_df.reset_index(inplace=True,drop=True)

    output=pd.DataFrame()
    for name in alliance_names:
        temp=temp_df.loc[:,temp_df.columns==name]
        output[name]=temp.apply(sum,axis=1)

    ## row of total
    output.loc["Total"]=output.apply(sum)
    output.fillna(0,inplace=True)
    
    ## insert a row of party names
    output=pd.concat([party_row,output],ignore_index = False)

    return output

from collections import defaultdict
def get_ally_party(data,alliance_dict):
    ally_party=defaultdict(list) ## make sure it's a list in each dict
    for party in data.loc[1,data.loc[1,]!="None"].str.upper():
        for alliance, parties in alliance_dict.items():
            for abbr, p in parties.items():
                if party in p:
                    if abbr not in ally_party[alliance]:
                        ally_party[alliance].append(abbr)
    return ally_party
   

# Year 2016

In [0]:
file="/content/drive/My Drive/Colab Notebooks/India_Election_Analytics/polling stations-odd/2016-odd.xlsx"
directory="/content/drive/My Drive/Colab Notebooks/India_Election_Analytics/polling stations-odd/ally_16_output/"

In [0]:
## dictionary for alliances who competed in 2016

alliance_names_16=["AIADMK+","DMK","PWF","NDA","PMK","NTK","Others"] ## unique alliances 

aiadmk_parties={"ADMK":["All India Anna Dravida Munnetra Kazhagam","ADMK","AIADMK","A.D.M.K."],
                "MJK":["Manithaneya Jananayaga Katchi","MJK","M.J.K."],
                "RPI":["Republican Party of India","RPI","R.P.I."],
                "SMK":["All India Samathuva Makhal Katchi","Samathuva Makkal Katchi","SMK","AISMK","S.M.K.","A.I.S.M.K."],
                "TNKIP":["Tamil Nadu Kongu Ilaingar Peravai","TNKIP","T.N.K.I.P."],
                "MP":["Mukkulathor Pulipadai","MP","M.P."],
                "TMML":["Tamil Maanila Muslim League","TMML","T.M.M.L."]}

aiadmk_upper={}
for key,value in aiadmk_parties.items():
    aiadmk_upper[key]=[x.upper() for x in value]

dmk_parties={"DMK":["Dravida Munnetra Kazhagam","DMK","D.M.K."],
             "INC":["Indian National Congress","INC","I.N.C."],
             "IUML":["Indian Union Muslim League","IUML","I.U.M.L."],
             "MAMAK":["Manithaneya Makkal Katchi","MAMAK","M.A.M.A.K."],
             "PT":["Puthiya Tamilakam","PT","P.T."],
             "PTMK":["Perunthalaivar Makkal Katchi","PTMK","P.T.M.K."],
             "TNPWP":["Tamil Nadu Peasants and Workers Party","TNPWP","T.N.P.W.P."],
             "SSP":["Samuga Samathuva Padai","SSP","S.S.P."]}

dmk_upper={}
for key,value in dmk_parties.items():
    dmk_upper[key]=[x.upper() for x in value]

pwf_parties={"DMDK":["Desiya Murpokku Dravida Kazhagam","DMDK","D.M.D.K."],
             "MDMK":["Marumalarchi Dravida Munnetra Kazhagam","MDMK","M.D.M.K."],
             "CPI":["Communist Party of India",\
             "Communist Party Of India Tamilaga Makkal Munnetra \
             Kazhagam Pattali Makkal Katchi Naam Tamilar Katchi","CPI","C.P.I."],
             "CPI(M)":["Communist Party of India (Marxist)","CPI(M)","CPIM","CPI (M)","C.P.I.(M)"],
             "TMC":["Tamil Manila Congress","TMC","TMC(M)","TMC (M)","T.M.C.(M)","T.M.C."],
             "VCK":["Viduthalai Chiruthaigal Katchi","VCK","V.C.K."]}
pwf_upper={}
for key,value in pwf_parties.items():
    pwf_upper[key]=[x.upper() for x in value]

nda_parties={"BJP":["Bharatiya Janata Party","BJP","B.J.P."],
             "IJK":["Indhiya Jananayaga Katchi","IJK","I.J.K."]}
nda_upper={}
for key,value in nda_parties.items():
    nda_upper[key]=[x.upper() for x in value]

pmk_party={"PMK":["Pattali Makkal Katchi","PMK","P.M.K."]} #independent 
pmk_upper={}
for key,value in pmk_party.items():
    pmk_upper[key]=[x.upper() for x in value]
           
ntk_party={"NTK":["Naam Tamilar Katchi","NTK","N.T.K."]} #independent
ntk_upper={}
for key,value in ntk_party.items():
    ntk_upper[key]=[x.upper() for x in value]

other_parties={"TNYP":["Tamilnadu Youth Party","TNYP","T.N.Y.P."],
        "KMDK":["KMDK","K.M.D.K."],
        "NOTA":["NONE OF THE ABOVE","NOTA"],
        #...
        "IND":["Independent","INDI","IND"]}
other_upper={}
for key,value in other_parties.items():
    other_upper[key]=[x.upper() for x in value]

alliance_dict_16=dict(zip(alliance_names_16,
                      [aiadmk_upper,dmk_upper,pwf_upper,nda_upper,pmk_upper,ntk_upper,other_upper]))

In [50]:
odd_const=[(2*x+1) for x in range(117)]

for constituency in odd_const:
    print(constituency)
    data1=read_data(constituency)
    output_df=get_ally_votes(data1,alliance_names_16,alliance_dict_16)
    output_df.to_csv(f"{directory}ally_16_{constituency}.csv")

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
221
223
225
227
229
231
233


## Year 2011

In [0]:
file="/content/drive/My Drive/Colab Notebooks/India_Election_Analytics/polling stations-odd/2011-odd.xlsx"
directory="/content/drive/My Drive/Colab Notebooks/India_Election_Analytics/polling stations-odd/ally_11_output/"

In [0]:
alliance_names_11=["AIADMK+","DMK","Others"] ## unique alliances 

aiadmk_parties={"ADMK":["All India Anna Dravida Munnetra Kazhagam","Anna Dravida Munnetra Kazhagam",\
                        "ADMK","AIADMK","A.D.M.K."],
                "DMDK":["Desiya Murpokku Dravida Kazhagam","DMDK",\
                "Desiya Murpokku Dravida Khazhagam (DMDK)","D.M.D.K."],
                "CPI":["Communist Party of India","Communist Party Of India Tamilaga Makkal Munnetra \
                Kazhagam Pattali Makkal Katchi Naam Tamilar Katchi",
                "CPI","C.P.I."],
                "CPI(M)":["Communist Party of India (Marxist)","CPI(M)","CPIM","CPI (M)","C.P.I.(M)"],
                "MAMAK":["Manithaneya Makkal Katchi","MAMAK","M.A.M.A.K."], ##???
                "PT":["Puthiya Tamilakam","PT","P.T."],
                "AIFB":["All India Forward Bloc","AIFB","A.I.F.B."],
                "SMK":["All India Samathuva Makhal Katchi","Samathuva Makkal Katchi",\
                "SMK","AISMK","S.M.K.","A.I.S.M.K."],
                "RPI":["Republican Party of India", "RPI","R.P.I."],
                "AIMMK":["All India Moovendar Munnani Kazhagam", "AIMMK","A.I.M.M.K."],
                "TNKIP":["Tamil Nadu Kongu Ilaingar Peravai","TNKIP","T.N.K.I.P."]}
aiadmk_upper={}
for key,value in aiadmk_parties.items():
    aiadmk_upper[key]=[x.upper() for x in value]

dmk_parties={"DMK":["Dravida Munnetra Kazhagam","DMK","Dravida Munnetra Kazhagam (DMK)","D.M.K."],
             "INC":["Indian National Congress","INC","I.N.C."],
             "PMK":["Pattali Makkal Katchi","PMK","P.M.K."],
             "IUML":["Indian Union Muslim League","IUML","I.U.M.L."],
             "KMK":["Kongunadu Munnetra Kazhagam","KMK","K.M.K."],
             "MMK":["Moovendar Munnetra Kazhagam","MMK","M.M.K."], ##???             
             "PTMK":["Perunthalaivar Makkal Katchi","PTMK","P.T.M.K."]}
dmk_upper={}
for key,value in dmk_parties.items():
    dmk_upper[key]=[x.upper() for x in value]

other_parties={"MDMK":["Marumalarchi Dravida Munnetra Kazhagam","MDMK","M.D.M.K."],
               "BJP":["Bharatiya Janata Party","BJP","Bharatiya Janata Party (BJP)","B.J.P."],
               "NOTA":["NONE OF THE ABOVE","NOTA"],
               "IJK":["Indiya Jananayaka Katchi (IJK)","Indiya Jananayaka Katchi","IJK","I.J.K."],
               #...
               "IND":["Independent","INDI","IND"]}
other_upper={}
for key,value in other_parties.items():
    other_upper[key]=[x.upper() for x in value]

alliance_dict_11=dict(zip(alliance_names_11,
                          [aiadmk_upper,dmk_upper,other_upper]))

In [53]:
odd_const=[(2*x+1) for x in range(117)]

for constituency in odd_const:

    print(constituency)
    data2=read_data(constituency)
    output_df=get_ally_votes(data2,alliance_names_11,alliance_dict_11)
    output_df.to_csv(f"{directory}ally_11_{constituency}.csv")

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
221
223
225
227
229
231
233


# Debug

In [0]:
## debug
#,73,93
constituency=95
data1=read_data(constituency)
alliances=get_alliance(data1,alliance_dict_11)

temp_df=pd.DataFrame()
temp_df=data1.loc[2:len(data1)-4,data1.loc[1,]!="None"]
temp_df.columns=alliances
temp_df.reset_index(inplace=True,drop=True)

output=pd.DataFrame()
for name in alliance_names:
    temp=temp_df.loc[:,temp_df.columns==name]
    print(temp)
    output[name]=temp.apply(sum,axis=1)

output.loc["Total"]=output.apply(sum)
output.fillna(0,inplace=True)
print(output)
# output_df=get_ally_votes(data1)